In [1]:
from platform import python_version
# я немного потанцевал с бубном, чтобы поставить версию питона пониже(нужна для tensorflow) и ничего не сломать, теперь начинаем лабу
print(python_version())

3.7.8


In [2]:
# видео карта подтягиваться не захотела, как и я не захотел заморачиваться для учебного примера
import numpy as np
from sklearn.datasets import load_digits
import tensorflow as tf

# данные по варианту
digits = load_digits()
digits['data'].shape

data = digits['data']
labels = digits['target']

data = data.reshape(data.shape[0],8,8,1)
print(data.shape, labels.shape)

(1797, 8, 8, 1) (1797,)


In [3]:
# данные из первой лабы + разделение на test train

def TTVsplit(X, Y, TrainLen=0.70, TestLen=0.2):
    if TrainLen+TestLen >= 1:
        return
    TrainLen = int(len(X)*TrainLen)
    TestLen = int(len(X)*TestLen)

    Xtrain = X[:TrainLen]
    Xtest = X[TrainLen:TrainLen+TestLen]
    Xval = X[TrainLen+TestLen:]
    Ytrain = Y[:TrainLen]
    Ytest = Y[TrainLen:TrainLen+TestLen]
    Yval = Y[TrainLen+TestLen:]
    
    return Xtrain, Xtest, Xval, Ytrain, Ytest, Yval

Xtrain, Xtest, Xval, Ytrain, Ytest, Yval = TTVsplit(data.copy(), labels.copy())
print(len(Xtrain)+len(Xtest)+len(Xval), len(data))

1797 1797


In [4]:
class Dataset(object):
    def __init__(self, X, y, batch_size, shuffle=False):
        """
        Construct a Dataset object to iterate over data X and labels y
        
        Inputs:
        - X: Numpy array of data, of any shape
        - y: Numpy array of labels, of any shape but with y.shape[0] == X.shape[0]
        - batch_size: Integer giving number of elements per minibatch
        - shuffle: (optional) Boolean, whether to shuffle the data on each epoch
        """
        assert X.shape[0] == y.shape[0], 'Got different numbers of data and labels'
        self.X, self.y = X, y
        self.batch_size, self.shuffle = batch_size, shuffle

    def __iter__(self):
        N, B = self.X.shape[0], self.batch_size
        idxs = np.arange(N)
        if self.shuffle:
            np.random.shuffle(idxs)
        return iter((self.X[i:i+B], self.y[i:i+B]) for i in range(0, N, B))


train_dset = Dataset(Xtrain, Ytrain, batch_size=64, shuffle=True)
val_dset = Dataset(Xval, Yval, batch_size=64, shuffle=False)
test_dset = Dataset(Xtest, Ytest, batch_size=64)

In [5]:
# тестовый пример из методички
class TwoLayerFC(tf.keras.Model):
    def __init__(self, hidden_size, num_classes):
        super(TwoLayerFC, self).__init__()        
        initializer = tf.initializers.VarianceScaling(scale=2.0)
        self.fc1 = tf.keras.layers.Dense(hidden_size, activation='relu',
                                   kernel_initializer=initializer)
        self.fc2 = tf.keras.layers.Dense(num_classes, activation='softmax',
                                   kernel_initializer=initializer)
        self.flatten = tf.keras.layers.Flatten()
    
    def call(self, x, training=False):
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.fc2(x)
        return x


def test_TwoLayerFC():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    x = tf.zeros((64, input_size))
    model = TwoLayerFC(hidden_size, num_classes)
    scores = model(x)
    print(scores.shape)
        
test_TwoLayerFC()

(64, 10)


Реализуйте трехслойную CNN для вашей задачи классификации.

Архитектура сети:

1. Сверточный слой (5 x 5 kernels, zero-padding = 'same')
2. Функция активации ReLU
3. Сверточный слой (3 x 3 kernels, zero-padding = 'same')
4. Функция активации ReLU
5. Полносвязный слой
6. Функция активации Softmax

In [6]:
# реализация первого задания
class ThreeLayerConvNet(tf.keras.Model):
    def __init__(self, channel_1, channel_2, num_classes):
        super(ThreeLayerConvNet, self).__init__()
        ########################################################################
        # TODO: Implement the __init__ method for a three-layer ConvNet. You   #
        # should instantiate layer objects to be used in the forward pass.     #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        self.conv1 = tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same")
        self.conv2 = tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same")
        
        self.flatten = tf.keras.layers.Flatten()
        self.fc1 = tf.keras.layers.Dense(num_classes, activation='softmax')

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################
        
    def call(self, x, training=False):
        scores = None
        ########################################################################
        # TODO: Implement the forward pass for a three-layer ConvNet. You      #
        # should use the layer objects defined in the __init__ method.         #
        ########################################################################
        # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.flatten(x)
        scores = self.fc1(x)

        # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
        ########################################################################
        #                           END OF YOUR CODE                           #
        ########################################################################        
        return scores

In [7]:
def test_ThreeLayerConvNet():    
    channel_1, channel_2, num_classes = 12, 8, 10
    model = ThreeLayerConvNet(channel_1, channel_2, num_classes)
    x = tf.zeros((64, 3, 32, 32))
    scores = model(x)
    print(scores.shape)

test_ThreeLayerConvNet()

(64, 10)


In [8]:
def train_part34(model_init_fn, optimizer_init_fn, num_epochs=1, is_training=False, print_every=5):
    """
    Simple training loop for use with models defined using tf.keras. It trains
    a model for one epoch on the CIFAR-10 training set and periodically checks
    accuracy on the CIFAR-10 validation set.
    
    Inputs:
    - model_init_fn: A function that takes no parameters; when called it
      constructs the model we want to train: model = model_init_fn()
    - optimizer_init_fn: A function which takes no parameters; when called it
      constructs the Optimizer object we will use to optimize the model:
      optimizer = optimizer_init_fn()
    - num_epochs: The number of epochs to train for
    
    Returns: Nothing, but prints progress during trainingn
    """    

    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    model = model_init_fn()
    optimizer = optimizer_init_fn()

    train_loss = tf.keras.metrics.Mean(name='train_loss')
    train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

    val_loss = tf.keras.metrics.Mean(name='val_loss')
    val_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='val_accuracy')

    t = 0
    for epoch in range(num_epochs):

        # Reset the metrics - https://www.tensorflow.org/alpha/guide/migration_guide#new-style_metrics
        train_loss.reset_states()
        train_accuracy.reset_states()

        for x_np, y_np in train_dset:
            with tf.GradientTape() as tape:

                # Use the model function to build the forward pass.
                scores = model(x_np, training=is_training)
                loss = loss_fn(y_np, scores)

                gradients = tape.gradient(loss, model.trainable_variables)
                optimizer.apply_gradients(zip(gradients, model.trainable_variables))

                # Update the metrics
                train_loss.update_state(loss)
                train_accuracy.update_state(y_np, scores)

                if t % print_every == 0:
                    val_loss.reset_states()
                    val_accuracy.reset_states()
                    for test_x, test_y in val_dset:
                        # During validation at end of epoch, training set to False
                        prediction = model(test_x, training=False)
                        t_loss = loss_fn(test_y, prediction)

                        val_loss.update_state(t_loss)
                        val_accuracy.update_state(test_y, prediction)

                    template = 'Iteration {}, Epoch {}, Loss: {}, Accuracy: {}, Val Loss: {}, Val Accuracy: {}'
                    print (template.format(t, epoch+1,
                                         train_loss.result(),
                                         train_accuracy.result()*100,
                                         val_loss.result(),
                                         val_accuracy.result()*100))
                t += 1

In [9]:
hidden_size, num_classes = 100, 10
learning_rate = 1e-2

def model_init_fn():
    return TwoLayerFC(hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10)

Iteration 0, Epoch 1, Loss: 15.266045570373535, Accuracy: 6.25, Val Loss: 11.106541633605957, Val Accuracy: 14.364640235900879
Iteration 5, Epoch 1, Loss: 8.279751777648926, Accuracy: 25.0, Val Loss: 5.2058329582214355, Val Accuracy: 44.19889450073242
Iteration 10, Epoch 1, Loss: 6.052606582641602, Accuracy: 31.960227966308594, Val Loss: 3.3001291751861572, Val Accuracy: 49.17127227783203
Iteration 15, Epoch 1, Loss: 4.7568182945251465, Accuracy: 41.6015625, Val Loss: 2.1563313007354736, Val Accuracy: 65.19337463378906
Iteration 20, Epoch 2, Loss: 1.5685036182403564, Accuracy: 71.875, Val Loss: 1.5655149221420288, Val Accuracy: 70.71823120117188
Iteration 25, Epoch 2, Loss: 0.6139066815376282, Accuracy: 84.89582824707031, Val Loss: 1.2553088665008545, Val Accuracy: 77.34806823730469
Iteration 30, Epoch 2, Loss: 0.5942705273628235, Accuracy: 84.09090423583984, Val Loss: 1.278493046760559, Val Accuracy: 76.24309539794922
Iteration 35, Epoch 2, Loss: 0.541005551815033, Accuracy: 85.15625,

Обучите трехслойную CNN. В tf.keras.optimizers.SGD укажите Nesterov momentum = 0.9.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/optimizers/SGD  

Значение accuracy на валидационной выборке после 1 эпохи обучения должно быть > 50%.  

In [10]:
# Done точность после первой эпохи больше 50% 
learning_rate = 3e-3
channel_1, channel_2, num_classes = 32, 16, 10

def model_init_fn():
    model = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    model = ThreeLayerConvNet(channel_1, channel_2, num_classes) 

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return model

def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10)

Iteration 0, Epoch 1, Loss: 2.8431551456451416, Accuracy: 18.75, Val Loss: 2.367692232131958, Val Accuracy: 18.232044219970703
Iteration 5, Epoch 1, Loss: 1.987907886505127, Accuracy: 36.71875, Val Loss: 1.2818776369094849, Val Accuracy: 69.6132583618164
Iteration 10, Epoch 1, Loss: 1.598429799079895, Accuracy: 51.42045593261719, Val Loss: 0.7685322761535645, Val Accuracy: 79.5580062866211
Iteration 15, Epoch 1, Loss: 1.2831567525863647, Accuracy: 61.62109375, Val Loss: 0.7430081963539124, Val Accuracy: 83.4254150390625
Iteration 20, Epoch 2, Loss: 0.5372335910797119, Accuracy: 85.9375, Val Loss: 0.4340353310108185, Val Accuracy: 85.08287048339844
Iteration 25, Epoch 2, Loss: 0.4097951352596283, Accuracy: 89.58332824707031, Val Loss: 0.5003913044929504, Val Accuracy: 86.1878433227539
Iteration 30, Epoch 2, Loss: 0.358908087015152, Accuracy: 90.48295593261719, Val Loss: 0.48282989859580994, Val Accuracy: 87.2928237915039
Iteration 35, Epoch 2, Loss: 0.3299383819103241, Accuracy: 91.2109

In [11]:
learning_rate = 1e-2

def model_init_fn():
    input_shape = (8, 8, 1)
    hidden_layer_size, num_classes = 100, 10
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    layers = [
        tf.keras.layers.Flatten(input_shape=input_shape),
        tf.keras.layers.Dense(hidden_layer_size, activation='relu',
                              kernel_initializer=initializer),
        tf.keras.layers.Dense(num_classes, activation='softmax', 
                              kernel_initializer=initializer),
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate) 

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 14.897880554199219, Accuracy: 6.25, Val Loss: 11.6000394821167, Val Accuracy: 13.259668350219727
Iteration 5, Epoch 1, Loss: 9.239715576171875, Accuracy: 15.104166984558105, Val Loss: 4.56206750869751, Val Accuracy: 28.176795959472656
Iteration 10, Epoch 1, Loss: 7.1071624755859375, Accuracy: 23.863636016845703, Val Loss: 3.50581955909729, Val Accuracy: 46.40884017944336
Iteration 15, Epoch 1, Loss: 5.724612236022949, Accuracy: 32.2265625, Val Loss: 2.499690294265747, Val Accuracy: 54.14365005493164


In [12]:
model = model_init_fn()
model.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=learning_rate),
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(Xtrain, Ytrain, batch_size=64, epochs=1, validation_data=(Xval, Yval))
model.evaluate(Xtest, Ytest)

12/12 [==============================] - 0s 637us/step - loss: 1.2916 - sparse_categorical_accuracy: 0.7159


[1.291609287261963, 0.7158774137496948]

Перепишите реализацию трехслойной CNN с помощью tf.keras.Sequential API . Обучите модель двумя способами.

In [13]:
def model_init_fn():
    model = None
    ############################################################################
    # TODO: Construct a three-layer ConvNet using tf.keras.Sequential.         #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                            END OF YOUR CODE                              #
    ############################################################################
    return model

learning_rate = 5e-4
def optimizer_init_fn():
    optimizer = None
    ############################################################################
    # TODO: Complete the implementation of model_fn.                           #
    ############################################################################
    # *****START OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****

    optimizer = tf.keras.optimizers.SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)

    # *****END OF YOUR CODE (DO NOT DELETE/MODIFY THIS LINE)*****
    ############################################################################
    #                           END OF YOUR CODE                               #
    ############################################################################
    return optimizer

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 3.376075506210327, Accuracy: 10.9375, Val Loss: 3.0269925594329834, Val Accuracy: 13.812154769897461
Iteration 5, Epoch 1, Loss: 2.8099453449249268, Accuracy: 17.96875, Val Loss: 2.310694932937622, Val Accuracy: 14.91712760925293
Iteration 10, Epoch 1, Loss: 2.560102939605713, Accuracy: 18.18181800842285, Val Loss: 2.0563642978668213, Val Accuracy: 33.70166015625
Iteration 15, Epoch 1, Loss: 2.3867783546447754, Accuracy: 21.6796875, Val Loss: 1.7848691940307617, Val Accuracy: 46.961326599121094


In [14]:
model = model_init_fn()
model.compile(optimizer='sgd',
              loss='sparse_categorical_crossentropy',
              metrics=[tf.keras.metrics.sparse_categorical_accuracy])
model.fit(Xtrain, Ytrain, batch_size=64, epochs=5, validation_data=(Xval, Yval))
model.evaluate(Xtest, Ytest)

Epoch 1/5
20/20 [==============================] - 1s 18ms/step - loss: 1.8130 - sparse_categorical_accuracy: 0.3989 - val_loss: 0.6415 - val_sparse_categorical_accuracy: 0.8287
Epoch 2/5
20/20 [==============================] - 0s 11ms/step - loss: 0.4284 - sparse_categorical_accuracy: 0.8972 - val_loss: 0.4193 - val_sparse_categorical_accuracy: 0.8895
Epoch 3/5
20/20 [==============================] - 0s 11ms/step - loss: 0.1969 - sparse_categorical_accuracy: 0.9525 - val_loss: 0.3497 - val_sparse_categorical_accuracy: 0.9006
Epoch 4/5
20/20 [==============================] - 0s 11ms/step - loss: 0.1611 - sparse_categorical_accuracy: 0.9502 - val_loss: 0.2942 - val_sparse_categorical_accuracy: 0.9171
Epoch 5/5
12/12 [==============================] - 0s 2ms/step - loss: 0.3140 - sparse_categorical_accuracy: 0.8997


[0.31401145458221436, 0.8997214436531067]

In [15]:
def two_layer_fc_functional(input_shape, hidden_size, num_classes):  
    initializer = tf.initializers.VarianceScaling(scale=2.0)
    inputs = tf.keras.Input(shape=input_shape)
    flattened_inputs = tf.keras.layers.Flatten()(inputs)
    fc1_output = tf.keras.layers.Dense(hidden_size, activation='relu',
                                 kernel_initializer=initializer)(flattened_inputs)
    scores = tf.keras.layers.Dense(num_classes, activation='softmax',
                             kernel_initializer=initializer)(fc1_output)

    # Instantiate the model given inputs and outputs.
    model = tf.keras.Model(inputs=inputs, outputs=scores)
    return model

def test_two_layer_fc_functional():
    """ A small unit test to exercise the TwoLayerFC model above. """
    input_size, hidden_size, num_classes = 50, 42, 10
    input_shape = (50,)
    
    x = tf.zeros((64, input_size))
    model = two_layer_fc_functional(input_shape, hidden_size, num_classes)
    
    scores = model(x)
    print(scores.shape)
        
test_two_layer_fc_functional()

(64, 10)


In [16]:
input_shape = (8, 8, 1)
hidden_size, num_classes = 100, 10
learning_rate = 1e-2

def model_init_fn():
    return two_layer_fc_functional(input_shape, hidden_size, num_classes)

def optimizer_init_fn():
    return tf.keras.optimizers.SGD(learning_rate=learning_rate)

train_part34(model_init_fn, optimizer_init_fn)

Iteration 0, Epoch 1, Loss: 13.2677640914917, Accuracy: 9.375, Val Loss: 9.635778427124023, Val Accuracy: 14.91712760925293
Iteration 5, Epoch 1, Loss: 7.625944137573242, Accuracy: 22.395832061767578, Val Loss: 3.6998202800750732, Val Accuracy: 41.98895263671875
Iteration 10, Epoch 1, Loss: 5.133203506469727, Accuracy: 37.21590805053711, Val Loss: 2.4154751300811768, Val Accuracy: 54.14365005493164
Iteration 15, Epoch 1, Loss: 3.9574062824249268, Accuracy: 46.38671875, Val Loss: 1.8586149215698242, Val Accuracy: 63.535911560058594


Поэкспериментируйте с архитектурой сверточной сети. Для вашего набора данных вам необходимо получить как минимум 70% accuracy на валидационной выборке за 10 эпох обучения. Опишите все эксперименты и сделайте выводы (без выполнения данного пункта работы приниматься не будут).  

Эспериментируйте с архитектурой, гиперпараметрами, функцией потерь, регуляризацией, методом оптимизации.  

https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/BatchNormalization#methods https://www.tensorflow.org/versions/r2.0/api_docs/python/tf/keras/layers/Dropout#methods

In [17]:
# РУБРИКА ЭКПЕРИМЕНТЫ
# в общем начну с модели сделанной в работе и попробую добить точность до ~97, как было на простых классификаторах

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

def optimizer_init_fn():
    learning_rate = 1e-3
    return tf.keras.optimizers.Adam(learning_rate) 

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 2.8871426582336426, Accuracy: 28.125, Val Loss: 2.40006422996521, Val Accuracy: 20.44198989868164
Iteration 20, Epoch 2, Loss: 0.404843270778656, Accuracy: 90.625, Val Loss: 0.5591102242469788, Val Accuracy: 86.7403335571289
Iteration 40, Epoch 3, Loss: 0.1670638769865036, Accuracy: 95.3125, Val Loss: 0.49945151805877686, Val Accuracy: 87.84529876708984
Iteration 60, Epoch 4, Loss: 0.12048865109682083, Accuracy: 96.875, Val Loss: 0.539639413356781, Val Accuracy: 88.39778900146484
Iteration 80, Epoch 5, Loss: 0.1582545042037964, Accuracy: 93.75, Val Loss: 0.5336039066314697, Val Accuracy: 88.39778900146484
Iteration 100, Epoch 6, Loss: 0.1266362965106964, Accuracy: 95.3125, Val Loss: 0.42647144198417664, Val Accuracy: 89.50276184082031
Iteration 120, Epoch 7, Loss: 0.061855778098106384, Accuracy: 98.4375, Val Loss: 0.3279155492782593, Val Accuracy: 92.81768035888672
Iteration 140, Epoch 8, Loss: 0.06636713445186615, Accuracy: 98.4375, Val Loss: 0.350859910249

In [18]:
# собтвенно точность 99/93 попробую нормализацию
# как бы я не растыкивал нормализацию эффекта ~0

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.466932535171509, Accuracy: 1.5625, Val Loss: 3.3626649379730225, Val Accuracy: 11.60221004486084
Iteration 20, Epoch 2, Loss: 0.35211580991744995, Accuracy: 87.5, Val Loss: 0.93828946352005, Val Accuracy: 70.1657485961914
Iteration 40, Epoch 3, Loss: 0.16350938379764557, Accuracy: 95.3125, Val Loss: 0.5835276246070862, Val Accuracy: 85.63536071777344
Iteration 60, Epoch 4, Loss: 0.08093790709972382, Accuracy: 98.4375, Val Loss: 0.4751368463039398, Val Accuracy: 90.05525207519531
Iteration 80, Epoch 5, Loss: 0.05999099463224411, Accuracy: 100.0, Val Loss: 0.39827847480773926, Val Accuracy: 90.60773468017578
Iteration 100, Epoch 6, Loss: 0.04303440824151039, Accuracy: 98.4375, Val Loss: 0.35022643208503723, Val Accuracy: 90.60773468017578
Iteration 120, Epoch 7, Loss: 0.03295818716287613, Accuracy: 100.0, Val Loss: 0.3148234784603119, Val Accuracy: 91.16021728515625
Iteration 140, Epoch 8, Loss: 0.0257298294454813, Accuracy: 100.0, Val Loss: 0.28798136115074

In [19]:
# собтвенно точность 99/93 попробую немного дропаут слои
# при различном rate - эффект такой же как от нормализации - никакой

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Dropout(rate=0.3),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.Dropout(rate=0.3),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.9931769371032715, Accuracy: 7.8125, Val Loss: 2.6365721225738525, Val Accuracy: 3.867403507232666
Iteration 20, Epoch 2, Loss: 1.212716817855835, Accuracy: 68.75, Val Loss: 1.1363452672958374, Val Accuracy: 81.21546936035156
Iteration 40, Epoch 3, Loss: 0.6065095067024231, Accuracy: 81.25, Val Loss: 0.5764067769050598, Val Accuracy: 85.08287048339844
Iteration 60, Epoch 4, Loss: 0.3889661431312561, Accuracy: 85.9375, Val Loss: 0.45435282588005066, Val Accuracy: 85.63536071777344
Iteration 80, Epoch 5, Loss: 0.24213647842407227, Accuracy: 93.75, Val Loss: 0.37123748660087585, Val Accuracy: 89.50276184082031
Iteration 100, Epoch 6, Loss: 0.20273646712303162, Accuracy: 92.1875, Val Loss: 0.3636649549007416, Val Accuracy: 90.05525207519531
Iteration 120, Epoch 7, Loss: 0.19075293838977814, Accuracy: 89.0625, Val Loss: 0.34534427523612976, Val Accuracy: 92.81768035888672
Iteration 140, Epoch 8, Loss: 0.14698359370231628, Accuracy: 93.75, Val Loss: 0.32755699753

In [20]:
# собтвенно точность 99/93 попробую немного дропаут слои
# еще поэкспериментирую с пулингом
# как сам пулинг, так и другие миксы с ним не дали особых результатов

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv2D(channel_2, 3, activation="relu", padding="same"),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.6793887615203857, Accuracy: 9.375, Val Loss: 3.46462082862854, Val Accuracy: 10.497238159179688
Iteration 20, Epoch 2, Loss: 0.46564823389053345, Accuracy: 90.625, Val Loss: 1.846177101135254, Val Accuracy: 33.149169921875
Iteration 40, Epoch 3, Loss: 0.19027945399284363, Accuracy: 93.75, Val Loss: 1.4553037881851196, Val Accuracy: 50.828731536865234
Iteration 60, Epoch 4, Loss: 0.1075747162103653, Accuracy: 98.4375, Val Loss: 0.7992900013923645, Val Accuracy: 76.24309539794922
Iteration 80, Epoch 5, Loss: 0.06554713845252991, Accuracy: 98.4375, Val Loss: 0.6055193543434143, Val Accuracy: 82.8729248046875
Iteration 100, Epoch 6, Loss: 0.05200958997011185, Accuracy: 98.4375, Val Loss: 0.5208153128623962, Val Accuracy: 87.2928237915039
Iteration 120, Epoch 7, Loss: 0.042390722781419754, Accuracy: 98.4375, Val Loss: 0.40499815344810486, Val Accuracy: 89.50276184082031
Iteration 140, Epoch 8, Loss: 0.03053460456430912, Accuracy: 100.0, Val Loss: 0.358177900314

In [21]:
# просто чек на то, что 10 эпох достаточно
train_part34(model_init_fn, optimizer_init_fn, num_epochs=20, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.910141706466675, Accuracy: 9.375, Val Loss: 3.1600263118743896, Val Accuracy: 11.60221004486084
Iteration 20, Epoch 2, Loss: 0.48923054337501526, Accuracy: 85.9375, Val Loss: 1.3396177291870117, Val Accuracy: 53.038673400878906
Iteration 40, Epoch 3, Loss: 0.26783299446105957, Accuracy: 92.1875, Val Loss: 1.0259990692138672, Val Accuracy: 65.19337463378906
Iteration 60, Epoch 4, Loss: 0.13305680453777313, Accuracy: 98.4375, Val Loss: 0.7247111201286316, Val Accuracy: 76.79557800292969
Iteration 80, Epoch 5, Loss: 0.0739607959985733, Accuracy: 96.875, Val Loss: 0.5561733841896057, Val Accuracy: 82.8729248046875
Iteration 100, Epoch 6, Loss: 0.05592239648103714, Accuracy: 98.4375, Val Loss: 0.4755704700946808, Val Accuracy: 85.08287048339844
Iteration 120, Epoch 7, Loss: 0.04841666668653488, Accuracy: 98.4375, Val Loss: 0.39910462498664856, Val Accuracy: 87.84529876708984
Iteration 140, Epoch 8, Loss: 0.03279917687177658, Accuracy: 100.0, Val Loss: 0.3638397

In [22]:
# упростим сетку, и посмотрим, что будет

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.2733516693115234, Accuracy: 10.9375, Val Loss: 2.3309926986694336, Val Accuracy: 15.469614028930664
Iteration 20, Epoch 2, Loss: 0.38070738315582275, Accuracy: 87.5, Val Loss: 0.4994008541107178, Val Accuracy: 87.84529876708984
Iteration 40, Epoch 3, Loss: 0.16293883323669434, Accuracy: 95.3125, Val Loss: 0.4834129810333252, Val Accuracy: 89.50276184082031
Iteration 60, Epoch 4, Loss: 0.12343704700469971, Accuracy: 95.3125, Val Loss: 0.4230926036834717, Val Accuracy: 90.05525207519531
Iteration 80, Epoch 5, Loss: 0.10338295996189117, Accuracy: 96.875, Val Loss: 0.3924087584018707, Val Accuracy: 88.95027923583984
Iteration 100, Epoch 6, Loss: 0.07324284315109253, Accuracy: 96.875, Val Loss: 0.3920191526412964, Val Accuracy: 91.16021728515625
Iteration 120, Epoch 7, Loss: 0.0702403336763382, Accuracy: 96.875, Val Loss: 0.40294113755226135, Val Accuracy: 91.16021728515625
Iteration 140, Epoch 8, Loss: 0.07577978819608688, Accuracy: 98.4375, Val Loss: 0.389212

In [23]:
# тут тоже не особо полезна нормализация

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 32, 16, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 3.1037650108337402, Accuracy: 4.6875, Val Loss: 2.594996929168701, Val Accuracy: 18.232044219970703
Iteration 20, Epoch 2, Loss: 0.23567846417427063, Accuracy: 90.625, Val Loss: 0.635598361492157, Val Accuracy: 81.21546936035156
Iteration 40, Epoch 3, Loss: 0.2023259997367859, Accuracy: 92.1875, Val Loss: 0.45637640357017517, Val Accuracy: 86.1878433227539
Iteration 60, Epoch 4, Loss: 0.07394493371248245, Accuracy: 96.875, Val Loss: 0.4075906276702881, Val Accuracy: 88.39778900146484
Iteration 80, Epoch 5, Loss: 0.08419841527938843, Accuracy: 98.4375, Val Loss: 0.3445611894130707, Val Accuracy: 91.16021728515625
Iteration 100, Epoch 6, Loss: 0.048150572925806046, Accuracy: 96.875, Val Loss: 0.3348959982395172, Val Accuracy: 91.71270751953125
Iteration 120, Epoch 7, Loss: 0.03562343120574951, Accuracy: 98.4375, Val Loss: 0.32161983847618103, Val Accuracy: 91.16021728515625
Iteration 140, Epoch 8, Loss: 0.02911796234548092, Accuracy: 98.4375, Val Loss: 0.31212

In [39]:
# еще некоторая куча тестов не дали результатов, куча разных вещей не давали вообще никакого результата, всегода +- 93
def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 128, 256, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid'),
        tf.keras.layers.Conv2D(channel_2, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 2.553586959838867, Accuracy: 7.8125, Val Loss: 7.661523342132568, Val Accuracy: 21.546960830688477
Iteration 20, Epoch 2, Loss: 0.3206251263618469, Accuracy: 92.1875, Val Loss: 0.4890342950820923, Val Accuracy: 87.2928237915039
Iteration 40, Epoch 3, Loss: 0.14770907163619995, Accuracy: 96.875, Val Loss: 0.34876036643981934, Val Accuracy: 87.84529876708984
Iteration 60, Epoch 4, Loss: 0.1437402069568634, Accuracy: 93.75, Val Loss: 0.4740939140319824, Val Accuracy: 87.84529876708984
Iteration 80, Epoch 5, Loss: 0.03163708373904228, Accuracy: 98.4375, Val Loss: 0.35470107197761536, Val Accuracy: 91.71270751953125
Iteration 100, Epoch 6, Loss: 0.07724542170763016, Accuracy: 98.4375, Val Loss: 0.3584446907043457, Val Accuracy: 90.60773468017578
Iteration 120, Epoch 7, Loss: 0.024498146027326584, Accuracy: 98.4375, Val Loss: 0.26789236068725586, Val Accuracy: 93.37016296386719
Iteration 140, Epoch 8, Loss: 0.01858612336218357, Accuracy: 98.4375, Val Loss: 0.32636

In [42]:
# изменения размерностей слоев тоже никуда не превели

def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 128, 64, 10
    layers = [
        tf.keras.layers.Conv2D(channel_1, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.MaxPool2D(pool_size=(2, 2), strides=(1, 1), padding='valid'),
        tf.keras.layers.Conv2D(channel_2, 5, activation="relu", padding="same", input_shape=input_shape),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=10, is_training=True, print_every=20)

Iteration 0, Epoch 1, Loss: 2.9153409004211426, Accuracy: 14.0625, Val Loss: 3.117932081222534, Val Accuracy: 18.784530639648438
Iteration 20, Epoch 2, Loss: 0.32756683230400085, Accuracy: 95.3125, Val Loss: 0.5580245852470398, Val Accuracy: 82.32044219970703
Iteration 40, Epoch 3, Loss: 0.11920501291751862, Accuracy: 93.75, Val Loss: 0.4073506295681, Val Accuracy: 89.50276184082031
Iteration 60, Epoch 4, Loss: 0.12326538562774658, Accuracy: 95.3125, Val Loss: 0.37492576241493225, Val Accuracy: 89.50276184082031
Iteration 80, Epoch 5, Loss: 0.0839371383190155, Accuracy: 98.4375, Val Loss: 0.49528130888938904, Val Accuracy: 89.50276184082031
Iteration 100, Epoch 6, Loss: 0.0971813052892685, Accuracy: 96.875, Val Loss: 0.3896768391132355, Val Accuracy: 92.26519775390625
Iteration 120, Epoch 7, Loss: 0.05998474732041359, Accuracy: 98.4375, Val Loss: 0.32715728878974915, Val Accuracy: 90.60773468017578
Iteration 140, Epoch 8, Loss: 0.04062815010547638, Accuracy: 98.4375, Val Loss: 0.276168

In [48]:
# ну и конечно же я помню результаты первых лаб о посему... толка оказалось 0
# продолжать так можно долго, но думаю, что я остановлюсь
def model_init_fn():
    input_shape = (8,8,1)
    channel_1, channel_2, num_classes = 128, 64, 10
    layers = [
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(num_classes, activation='softmax')
    ]
    model = tf.keras.Sequential(layers)
    return model

train_part34(model_init_fn, optimizer_init_fn, num_epochs=100, is_training=True, print_every=100)

Iteration 0, Epoch 1, Loss: 10.436516761779785, Accuracy: 4.6875, Val Loss: 9.603763580322266, Val Accuracy: 6.077348232269287
Iteration 100, Epoch 6, Loss: 0.21325251460075378, Accuracy: 95.3125, Val Loss: 0.7091392874717712, Val Accuracy: 86.1878433227539
Iteration 200, Epoch 11, Loss: 0.1154417023062706, Accuracy: 96.875, Val Loss: 0.5377813577651978, Val Accuracy: 88.95027923583984
Iteration 300, Epoch 16, Loss: 0.08661089837551117, Accuracy: 96.875, Val Loss: 0.4578511714935303, Val Accuracy: 91.16021728515625
Iteration 400, Epoch 21, Loss: 0.06255783885717392, Accuracy: 96.875, Val Loss: 0.424409955739975, Val Accuracy: 91.71270751953125
Iteration 500, Epoch 26, Loss: 0.04466624557971954, Accuracy: 98.4375, Val Loss: 0.41194331645965576, Val Accuracy: 92.26519775390625
Iteration 600, Epoch 31, Loss: 0.03330942243337631, Accuracy: 98.4375, Val Loss: 0.40820053219795227, Val Accuracy: 92.81768035888672
Iteration 700, Epoch 36, Loss: 0.025487396866083145, Accuracy: 100.0, Val Loss: 